In [141]:
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore


# Utility function to ensure directory existence
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory, exist_ok=True)

# Step 1: Data preparation and random label selection
def read_and_prepare_data(input_file_path, num_random_label):
    files = sorted(glob.glob(input_file_path))
    data_info = []
    
    for file_ in files:
        dataset_name = file_.split('/')[-2]
        print("Processing:", dataset_name)
        df = pd.read_csv(file_, sep='\t', low_memory=False)
        max_num_hvg_path = os.path.join(file_.rsplit('/', 2)[0], dataset_name, "max_num_hvg.txt")
        
        with open(max_num_hvg_path, 'r') as file:
            max_num_hvg = int(file.read().strip())

        df['num_hvg'] = df['num_hvg'].apply(lambda val: max_num_hvg if not str(val).isdigit() else int(val))

        df['num_hvg'] = pd.Categorical(df['num_hvg'], categories=sorted(set(df['num_hvg'].unique().tolist() + [max_num_hvg])), ordered=True)
        
        labels = np.random.choice(df['label'].unique(), size=min(num_random_label, df['label'].nunique()), replace=False)
        data_info.append((dataset_name, df, labels))
    
    return data_info


def process_dataset(dataset_info, output_base_path, num_random_label, avg_zscore_data):
    dataset_name, df, labels = dataset_info
    
    combinations = df[['num_pcs', 'num_nn', 'hvg_norm_combo']].drop_duplicates().to_dict('records')

    combo_avg_data = {}
    
    for combo in combinations:
        # Filtering dataframe based on current combination
        combo_df = df[
            (df['num_pcs'] == combo['num_pcs']) & 
            (df['num_nn'] == combo['num_nn']) & 
            (df['hvg_norm_combo'] == combo['hvg_norm_combo'])
        ]
        
        combo_key = (combo['num_pcs'], combo['num_nn'], combo['hvg_norm_combo'])
        combo_dir = os.path.join(output_base_path, f"PCs_{combo['num_pcs']}_K_{combo['num_nn']}_Combo_{combo['hvg_norm_combo']}")
        dataset_dir = os.path.join(combo_dir, dataset_name)
        ensure_dir(dataset_dir)
        
        # Create a DataFrame to store aggregated data
        aggregated_df = pd.DataFrame()
        
        for label in labels:
            label_df = combo_df[combo_df['label'] == label].copy()
            label_dir = os.path.join(dataset_dir, label)
            ensure_dir(label_dir)

            # Apply z-score normalization for 'neighbor_enrichment'
            if 'zscore_neighbor_enrichment' not in label_df.columns:
                label_df['zscore_neighbor_enrichment'] = zscore(label_df['neighbor_enrichment'])

            # Ensure 'zscore_neighbor_enrichment' column exists
            assert 'zscore_neighbor_enrichment' in label_df.columns, "'zscore_neighbor_enrichment' column missing from label_df"

            # Check and generate raw plot
            raw_plot_path = os.path.join(label_dir, "raw.png")
            if not os.path.exists(raw_plot_path):
                plt.figure()
                sns.lineplot(x="num_hvg", y="neighbor_enrichment", data=label_df, marker='o')
                plt.title(f'Perturb: {label} PCs: {combo["num_pcs"]} K: {combo["num_nn"]} Combo: {combo["hvg_norm_combo"]} {dataset_name}')
                plt.savefig(raw_plot_path)
                plt.close()
            
            # Check and generate z-score plot
            zscore_plot_path = os.path.join(label_dir, "zscore.png")
            if not os.path.exists(zscore_plot_path):
                label_df['zscore_neighbor_enrichment'] = zscore(label_df['neighbor_enrichment'])
                plt.figure()
                sns.lineplot(x="num_hvg", y="zscore_neighbor_enrichment", data=label_df, marker='o')
                plt.title(f'Perturb: {label} PCs: {combo["num_pcs"]} K: {combo["num_nn"]} Combo: {combo["hvg_norm_combo"]} {dataset_name}')
                plt.savefig(zscore_plot_path)
                plt.close()
            
            # Aggregate data for the combination plot
            label_df['label'] = label  # Add back the label column for aggregation

            aggregated_df = pd.concat([aggregated_df, label_df], ignore_index=True)
        
        # Generate aggregated plot after updating all label DataFrames
        if not aggregated_df.empty:
            aggregated_plot_path = os.path.join(dataset_dir, "aggregate over random x labels.png")
            if not os.path.exists(aggregated_plot_path):
                plt.figure()
                sns.lineplot(x="num_hvg", y="zscore_neighbor_enrichment", hue="label", data=aggregated_df, marker='o')
                plt.title(f'PCs: {combo["num_pcs"]} K: {combo["num_nn"]} Combo: {combo["hvg_norm_combo"]} {dataset_name}')
                plt.savefig(aggregated_plot_path)
                plt.close()
            
            # After aggregation, calculate the average z-score for each 'num_hvg'
            avg_zscore_df = aggregated_df.groupby('num_hvg')['zscore_neighbor_enrichment'].mean().reset_index(name='avg_zscore_neighbor_enrichment_across_random_x_labels')
            # Generate plot for average z-score across labels
            average_plot_path = os.path.join(dataset_dir, "average over random x labels.png")
            if not os.path.exists(average_plot_path):
                plt.figure()
                sns.lineplot(x="num_hvg", y="avg_zscore_neighbor_enrichment_across_random_x_labels", data=avg_zscore_df, marker='o')
                plt.title(f'PCs: {combo["num_pcs"]} K: {combo["num_nn"]} Combo: {combo["hvg_norm_combo"]} {dataset_name}')
                plt.savefig(average_plot_path)
                plt.close()


            # Store the average data for later aggregation over datasets
            if combo_key not in avg_zscore_data:
                avg_zscore_data[combo_key] = {}
            avg_zscore_data[combo_key][dataset_name] = avg_zscore_df


    
# Aggregate and generate plots over datasets
def aggregate_over_datasets(output_base_path, avg_zscore_data, avg_zscore_labels_datasets):
    for combo_key, datasets_avg_data in avg_zscore_data.items():
        num_pcs, num_nn, hvg_norm_combo = combo_key
        combo_dir = os.path.join(output_base_path, f"PCs_{num_pcs}_K_{num_nn}_Combo_{hvg_norm_combo}")
        ensure_dir(combo_dir)
        
        all_datasets_avg_df = pd.concat(datasets_avg_data.values(), keys=datasets_avg_data.keys(), names=['Dataset', 'Index']).reset_index(level='Dataset')
        
        aggregate_plot_path = os.path.join(combo_dir, "aggregate over datasets.png")
        if not os.path.exists(aggregate_plot_path):
            plt.figure()
            sns.lineplot(x="num_hvg", y="avg_zscore_neighbor_enrichment_across_random_x_labels", hue="Dataset", data=all_datasets_avg_df, marker='o')
            plt.title(f'PCs: {num_pcs} K: {num_nn} Combo: {hvg_norm_combo}')
            plt.savefig(aggregate_plot_path)
            plt.close()

        # Calculate the average z-score across all datasets for each 'num_hvg'
        average_across_datasets_df = all_datasets_avg_df.groupby('num_hvg')['avg_zscore_neighbor_enrichment_across_random_x_labels'].mean().reset_index(name='avg_zscore_across_labels&datasets')
        
        # Plot showing the average z-score across all datasets
        average_plot_path = os.path.join(combo_dir, "average over datasets.png")
        if not os.path.exists(average_plot_path):
            plt.figure()
            sns.lineplot(x="num_hvg", y="avg_zscore_across_labels&datasets", data=average_across_datasets_df, marker='o')
            plt.title(f'PCs: {num_pcs} K: {num_nn} Combo: {hvg_norm_combo}')
            plt.savefig(average_plot_path)
            plt.close()

        if combo_key not in avg_zscore_labels_datasets:
            avg_zscore_labels_datasets[combo_key] = {}
        avg_zscore_labels_datasets[combo_key] = average_across_datasets_df



def aggregate_over_combinations(output_base_path, avg_zscore_data):
    all_data_combo_df = pd.DataFrame()
    for combo_key, datasets_avg_data in avg_zscore_data.items():
        num_pcs, num_nn, hvg_norm_combo = combo_key
        combo_label = f'PCs: {num_pcs} K: {num_nn} Combo: {hvg_norm_combo}'

        # Collect data from all datasets for the current combination
        for dataset_name, avg_data in datasets_avg_data.items():
            # Ensure avg_data is a DataFrame
            if isinstance(avg_data, pd.DataFrame):
                avg_data['Combo'] = combo_label  # This labels every row in the DataFrame with the combo information
                avg_data['Dataset'] = dataset_name  # Also label rows with the dataset name
                # Now concatenate this updated DataFrame to the aggregated DataFrame
                all_data_combo_df = pd.concat([all_data_combo_df, avg_data], ignore_index=True)
            else:
                print(f"Expected a DataFrame for dataset {dataset_name}, but received {type(avg_data)}")
    # Plotting each dataset with all combinations
    for dataset_name, group_df in all_data_combo_df.groupby('Dataset'):
        # Determine path and save the plot
        dataset_plot_path = os.path.join(output_base_path, dataset_name, "aggregate over PCs, K, and norm methods.png")
        ensure_dir(os.path.join(output_base_path, dataset_name))  # Ensure the directory exists
        if not os.path.exists(dataset_plot_path):
            plt.figure(figsize=(20, 12))
            sns.lineplot(x="num_hvg", y="avg_zscore_neighbor_enrichment_across_random_x_labels", hue="Combo", data=group_df, marker='o')
            plt.title(f'{dataset_name}')
            plt.legend(title='Combo', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize='small')
            plt.tight_layout(rect=[0, 0, 0.75, 1])
            plt.savefig(dataset_plot_path)
            plt.close()


def aggregate_and_plot_combinations(output_base_path, avg_zscore_labels_datasets):
    final_aggregated_df = pd.DataFrame()
    
    for combo_key, datasets_avg_data in avg_zscore_labels_datasets.items():
        num_pcs, num_nn, hvg_norm_combo = combo_key
        combo_label = f'PCs: {num_pcs} K: {num_nn} Combo: {hvg_norm_combo}'

        # Here, datasets_avg_data should already be a DataFrame
        # Add the 'Combo' label directly to this DataFrame
        datasets_avg_data['Combo'] = combo_label  # This labels every row in the DataFrame with the combo information
        
        # Now concatenate this updated DataFrame to the final aggregated DataFrame
        final_aggregated_df = pd.concat([final_aggregated_df, datasets_avg_data], ignore_index=True)


    final_plot_path = os.path.join(output_base_path, "PC_K_norm_combined_effect_on_num_HVG_across_datasets.png")
    if not os.path.exists(final_plot_path):
        # Plot showing different lines for different parameter combinations
        plt.figure(figsize=(20, 12))
        ax = sns.lineplot(x="num_hvg", y="avg_zscore_across_labels&datasets", hue="Combo", data=final_aggregated_df, marker='o', errorbar=None)
        plt.title('PC_K_norm__combined_effect_on_num_HVG_across_datasets')

        # Place the legend outside the figure/plot
        plt.legend(title='Combo', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize='small')

        # Ensure the plot lays out correctly with the legend outside
        plt.tight_layout(rect=[0, 0, 0.85, 1])
        
        plt.savefig(final_plot_path)
        plt.close() 

      
# Main function to orchestrate the process
def main(input_file_path, output_base_path, num_random_label):
    datasets_info = read_and_prepare_data(input_file_path, num_random_label)
    
    avg_zscore_data = {}
    for dataset_info in datasets_info:
        process_dataset(dataset_info, output_base_path, num_random_label, avg_zscore_data)

    avg_zscore_labels_datasets = {}
    # After processing all datasets, aggregate over datasets and generate plots
    aggregate_over_datasets(output_base_path, avg_zscore_data, avg_zscore_labels_datasets)


    # Also aggregate over combos and generate plots
    aggregate_over_combinations(output_base_path, avg_zscore_data)
    
    # Aggregate across combinations and generate the final plot
    aggregate_and_plot_combinations(output_base_path, avg_zscore_labels_datasets)


In [148]:
# Breast cancer data
input_file_path = "/Genomics/pritykinlab/yujie/preprocessing_benchmarking/results/breast_cancer/qian/num_HVG/*/aggregated_results.tsv"
output_base_path = '/Genomics/pritykinlab/yujie/preprocessing_benchmarking/analysis/plots/breast_cancer/HVG_trend/'
num_random_label = 5  # Number of labels to randomly select

main(input_file_path, output_base_path, num_random_label)

Processing: patient_41_adata
Processing: patient_42_adata
Processing: patient_43_adata
Processing: patient_44_adata
Processing: patient_45_adata
Processing: patient_46_adata
Processing: patient_47_adata
Processing: patient_48_adata
Processing: patient_49_adata
Processing: patient_50_adata
Processing: patient_51_adata
Processing: patient_52_adata
Processing: patient_53_adata
Processing: patient_54_adata


In [149]:
# scPerturb
input_file_path = "/Genomics/pritykinlab/yujie/preprocessing_benchmarking/results/harmonized_perturb_seq/num_HVG/*/aggregated_results.tsv"
output_base_path = '/Genomics/pritykinlab/yujie/preprocessing_benchmarking/analysis/plots/harmonized_perturb_seq/HVG_trend/'
num_random_label = 5  # Number of labels to randomly select

main(input_file_path, output_base_path, num_random_label)

Processing: AdamsonWeissman2016_GSM2406675_10X001
Processing: AdamsonWeissman2016_GSM2406677_10X005
Processing: AdamsonWeissman2016_GSM2406681_10X010
Processing: AissaBenevolenskaya2021
Processing: ChangYe2021
Processing: DatlingerBock2017
Processing: DatlingerBock2021
Processing: DixitRegev2016
Processing: NormanWeissman2019_filtered
Processing: PapalexiSatija2021_eccite_RNA
Processing: PapalexiSatija2021_eccite_arrayed_RNA
Processing: SchiebingerLander2019_GSE106340
Processing: ShifrutMarson2018
Processing: SrivatsanTrapnell2020_sciplex2
Processing: SrivatsanTrapnell2020_sciplex4
Processing: TianKampmann2019_day7neuron
Processing: TianKampmann2021_CRISPRa
Processing: TianKampmann2021_CRISPRi
Processing: XieHon2017
